# Short tour of Julia for GPU programming and Machine Learning

[Julia](https://julialang.org/) is a scientific programming language that is free and open source for downloads, documentation, learning resources etc. Bridging high-level interpreted and low-level compiled languages, it offers high performance (comparable to C and Fortran) without sacrificing simplicity and programming productivity (like in Python or R).

Julia has a rich ecosystem of libraries aimed towards scientific computing and a powerful built-in package manager to install and manage their dependencies. Julia is also gaining ground in HPC as it supports both multithreaded and distributed-memory parallelisation, as well as GPU computing.

ENCCS provides learning materials for Julia programming:
- [Introduction to programming in Julia](https://enccs.github.io/julia-intro/)
- [Julia for High-Performance Data Analytics](https://enccs.github.io/julia-for-hpda/)
- [Julia for High-Performance Scientific Computing](https://enccs.github.io/julia-for-hpc/)

## Julia was created to solve the two-language problem

To run code in any programming language, some sort of translation into machine instructions needs to take place, but how this translation takes place differs between programming languages:
- Interpreted languages like Python and R translate instructions line by line.
- Compiled languages like C/C++ and Fortran are translated by a compiler prior to program execution.

The benefits of interpreted languages are that they are easier to read and write because less information on aspects like types and array sizes needs to be provided. Programmer productivity is thus higher in interpreted languages, but compiled languages can perform faster by orders of magnitude because the compiler can perform optimizations during the translation to assembly. This is also known as the *two-language problem*.

In many ways Julia looks like an interpreted language, and mostly behaves like one. But before each function is executed, Julia’s LLVM compiler will compile it `“just in time” (JIT)`. More on that later. Thus you get the flexibility of an interpreted language and the execution speed of the compiled language at the cost of waiting a bit longer for the first execution of any function.

Julia has been designed to be both fast and dynamic. In the words of its developers: 

> We want a language that’s open source, with a liberal license. We want the speed of C with the dynamism of Ruby. We want a language that’s homoiconic, with true macros like Lisp, but with obvious, familiar mathematical notation like Matlab. We want something as usable for general programming as Python, as easy for statistics as R, as natural for string processing as Perl, as powerful for linear algebra as Matlab, as good at gluing programs together as the shell. Something that is dirt simple to learn, yet keeps the most serious hackers happy. We want it interactive and we want it compiled. (Did we mention it should be as fast as C?).

## Julia Micro-Benchmarks

<div>
<img src="https://julialang.org/assets/images/benchmarks.svg", width="800"/>
</div>

# 1. Basic syntax

## Scalars

In [1]:
A = 3.14
println(A, " --- ", typeof(A))

B = 10
println(B, " --- ", typeof(B))

C = true
println(C, " --- ", typeof(C))

D = 3+4im
println(D, " --- ", typeof(D))

E = "Hello, Julia"
println(E, " --- ", typeof(E))

# supertypes and subtypes
print(supertypes(Float64), " --- ", subtypes(Int64))

3.14 --- Float64
10 --- Int64
true --- Bool
3 + 4im --- Complex{Int64}
hello, Julia --- String
(Float64, AbstractFloat, Real, Number, Any) --- Type[]

## Vectors and Arrays

In [9]:
a1 = [1, 2, 3, 4] # 4-element Vector{Int64}
a2 = [i^3 for i in [1,2,3,4]] # Array comprehension

m1 = [[1,2]  [4,5]  [7,8]] # 2×3 Matrix{Int64}
m2 = zeros(4,4,4,4) # Zero 4×4×4×4 Array{Float64, 4}

# broadcasting
b1 = a1.^2
b2 = a2 .- a1

4-element Vector{Int64}:
  0
  6
 24
 60

## Loops and Conditionals

`for` loops iterate over iterables, including types like `Range`, `Array`, `Set` and `Dict`.

In [10]:
for i in [1,2,3,4,5]
    println("i = $i")
end

i = 1
i = 2
i = 3
i = 4
i = 5


In [11]:
A = [1 2; 3 4]
# visit each index of A efficiently
for i in eachindex(A)
    println("i = $i, A[i] = $(A[i])")
end

i = 1, A[i] = 1
i = 2, A[i] = 3
i = 3, A[i] = 2
i = 4, A[i] = 4


In [12]:
for (k, v) in Dict("A" => 1, "B" => 2, "C" => 3)
    println("$k is $v")
end

B is 2
A is 1
C is 3


In [ ]:
x = 6
if x > 5
    println("x > 5")
elseif x < 5    # optional elseif
    println("x < 5")
else            # optional else
    println("x = 5")
end

In [ ]:
n = 0
while n < 10
    n += 1
    println(n)
end

## Working with files

In [13]:
open("myfile.txt", "w") do f
    write(f, "a line")
end

open("myfile.txt") do f
    # read from file
    lines = readlines(f)
    println(lines)
end

["another line"]


## Functions

In [14]:
function f(x,y)
    x + y   # can also use "return" keyword
end

f(3, 5)

8

In [ ]:
## Keyword arguments can be added after `;`

function greet_dog(; greeting = "Hi", dog_name = "Fido")  # note the ;
    println("$greeting $dog_name")
end

greet_dog(dog_name = "Coco", greeting = "Go fetch")

In [ ]:
# Optional arguments

function date(y, m=2, d=2)
    month = lpad(m, 2, "0")  # lpad pads from the left
    day = lpad(d, 2, "0")
    println("$y-$month-$day")
end

date(2024)
date(2024, 3)
date(2024, 3, 3)

In [ ]:
# argument types can be specified explicitly
function f(x::Float64, y::Float64)
    return x*y
end

# return types can also be specified
function g(x, y)::Int8
    return x * y
end

# 2. Special features of Julia

## Hierarchy Types

Julia is a dynamically typed language and does not require users to explicitly declare types because types are inferred and used at runtime. The sophisticated type system helps Julia to generate efficient code.

As types play a fundamental role in Julia’s design, it’s important to have a mental model of Julia’s type system. There are two basic kinds of types in Julia:
- Abstract types which define the kind of a thing, that is, represent sets of related types.
- Concrete types which describe data structures, that is, concrete implementations that can be used for variables.

Furthermore, a **primitive type** consists of plain bits such as an integer, character or floating point number. A **parametric type** represents a set of types. Types in Julia form a “type tree”, in which the leaves are concrete types.

<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Julia-number-type-hierarchy.svg/1920px-Julia-number-type-hierarchy.svg.png" width="800"/>
</div>

## Functions and Methods

In [16]:
function sumsquare(x, y)
    return x^2 + y^2
end

sumsquare (generic function with 1 method)

Each function can have multiple methods. A method is a function defined for specific arguments types.
Here we define methods using short form syntax.

In [17]:
sumsquare(x::Float64, y::Float64) = x^2 + y^2
sumsquare(x::Int64, y::Int64) = x^2 + y^2
sumsquare(x::Complex{Float64}, y::Complex{Float64}) = x^2 + y^2

sumsquare (generic function with 4 methods)

Julia’s type system also enables `multiple dispatch`, that is, choosing the most specific method of a function based on the argument types. 

`Multiple dispatch` sets the language apart from most other languages and makes it composable and fast when combined with just-in-time (JIT) compilation using the LLVM compiler toolchain.

In [ ]:
# Passing arguments with all kinds of types to this function

# Int64
println(sumsquare(2, 3))

# Float64
println(sumsquare(2.72, 3.83))
    
# Complex{Float64}
println(sumsquare(1.2+2.3im, 2.1-1.5im))

We can also create a composite type (a struct), and create a new method for that type. The `Point` composite type below is furthermore *parametric*, where we restrict the type T to be a subtype of `Real`:


In [18]:
struct Point{T<:Real}
    x::T
    y::T
end

We can now create two variables of type `Point`, and define a method for `sumsquare` which accepts this type:

In [19]:
function sumsquare(p1::Point, p2::Point)
   return Point(p1.x^2 + p2.x^2, p1.y^2 + p2.y^2)
end

p1, p2 = Point(1.0, 2.0), Point(2.0, 3.0)
sumsquare(p1, p2)  

Point{Float64}(5.0, 13.0)

## Code generation

Julia was designed from the beginning for HPC and this is accomplished by compiling Julia programs to efficient native code for multiple platforms via the `LLVM` compiler toolchain and `JIT` compilation. The Julia runtime code generator produces an LLVM **Intermediate Representation** (IR) which the LLMV compiler then converts to machine code using sophisticated optimization technology
- Interpreted languages rely on a runtime which directly executes the source code.
- Compiled languages rely on ahead-of-time compilation where source code is converted to an executable before execution.
- JIT compilation is when code is compiled to machine code at runtime.

<div>
<img src="https://enccs.github.io/julia-intro/_images/julia-code-generation.png" width="800"/>
</div>

To see the various forms of lowered code that is generated by the JIT compiler we can use several macros. Inspecting the lowered form for functions requires selection of the specific method to display, because generic functions may have many methods with different type signatures.

In [20]:
# Surface level AST
Meta.parse("sumsquare(1, 2)") |> dump
Meta.parse("sumsquare(p1, p2)") |> dump

# Lowered form of AST
@code_lowered sumsquare(1, 2)
@code_lowered sumsquare(p1, p2)

# Type-inferred lowered form of AST
@code_typed sumsquare(1, 2)
@code_typed sumsquare(1.0, 2.0)
@code_typed sumsquare(p1, p2)

# Lowered and type-inferred ASTs
@code_warntype sumsquare(1.0, 2.0)
@code_warntype sumsquare(p1, p2)

# LLVM intermediate representation:
@code_llvm sumsquare(1, 2)
@code_llvm sumsquare(1.0, 2.0)
@code_llvm sumsquare(p1, p2)

# native assembly instructions:
@code_native sumsquare(1, 2)
@code_native sumsquare(1.0, 2.0)
@code_native sumsquare(p1, p2)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol sumsquare
    2: Int64 1
    3: Int64 2
Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol sumsquare
    2: Symbol p1
    3: Symbol p2
MethodInstance for sumsquare(::Float64, ::Float64)
  from sumsquare(x::Float64, y::Float64) @ Main ~/repos/lessons/webinar_documents/2024-feb-02-webinar/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X42sdnNjb2RlLXJlbW90ZQ==.jl:1
Arguments
  #self#::Core.Const(sumsquare)
  x::Float64
  y::Float64
Body::Float64
1 ─ %1 = Main.:^::Core.Const(^)
│   %2 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %3 = (%2)()::Core.Const(Val{2}())
│   %4 = Base.literal_pow(%1, x, %3)::Float64
│   %5 = Main.:^::Core.Const(^)
│   %6 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %7 = (%6)()::Core.Const(Val{2}())
│   %8 = Base.literal_pow(%5, y, %7)::Float64
│   %9 = (%4 + %8)::Float64
└──      return %9

MethodInstance for sumsquare(::Point{Float64}, ::Point{Float64})
  from sumsq

## Metaprogramming

Julia represents its own code as a data structure accessible from the language itself. Since code is represented by objects that can be created and manipulated from within the language, it is possible for a program to transform and generate its own code, that is to create powerful macros (the term "metaprogramming" refers to the possibility to write code that writes code that is then evaluated).

Note the difference from C or C++ macros. There, macros work performing textual manipulation and substitution before any actual parsing or interpretation occurs.

In Julia, macros work when the code has already been parsed and organised in a syntax tree, and hence the semantic is much richer and allows for much more powerful manipulations.

More reading materials for the metaprogramming
- [Documentation on metaprogramming](https://docs.julialang.org/en/v1/manual/metaprogramming/)
- [Metaprogramming tutorial from JuliaCon21](https://github.com/dpsanders/Metaprogramming_JuliaCon_2021)

# 3. GPU programming using Julia

Julia has first-class support for GPU programming through the following packages that target GPUs from all major vendors:
- [CUDA.jl](https://cuda.juliagpu.org/stable/) for NVIDIA GPUs
- [AMDGPU.jl](https://amdgpu.juliagpu.org/stable/) for AMD GPUs
- [oneAPI.jl](https://github.com/JuliaGPU/oneAPI.jl) for Intel GPUs
- [Metal.jl](https://github.com/JuliaGPU/Metal.jl) for Apple M-series GPUs

ENCCS reading materials:
- [GPU Programming: When, Why and How?](https://enccs.github.io/gpu-programming/6-language-support/#julia)
- [Julia for High-Performance Scientific Computing](https://enccs.github.io/julia-for-hpc/GPU/)
- [Julia for High-Performance Data Analytics](https://enccs.github.io/julia-for-hpda/)

Moreover, the [KernelAbstractions.jl](https://juliagpu.github.io/KernelAbstractions.jl/stable/) package can be useful to write vendor-agnostic code that can execute on different GPU brands (and also fallback on CPUs if necessary).

## Setup and Access to GPUs

In [ ]:
# I have an AMDGPU so I need the AMDGPU.jl package
using Pkg
Pkg.add("KernelAbstractions")
Pkg.add("AMDGPU")

## The Array interface

GPU programming with Julia can be as simple as using a different array type instead of regular Base.Array. 

For Apple M-series GPUs, it should use `ROCArray` from `AMDGPU.jl`.

In [ ]:
using AMDGPU

# copy an array to GPU and executes a simple operation on GPU
A_d = ROCArray([1,2,3,4])
A_d .+= 1

# fetch data from GPU to CPU
A = Array(A_d) # the overhead of copying data to GPU makes such simple calculations very slow

In [ ]:
# A code example for matrix multiplication
using BenchmarkTools
using AMDGPU

A = rand(Float32, 2^9, 2^9); # Float64 does not work for Apple GPU
A_d = ROCArray(A);

@btime $A * $A;

@btime AMDGPU.@sync $A_d * $A_d;

## Writing your own kernels

In [ ]:
function vadd!(C, A, B)
    for i in 1:length(A)
        @inbounds C[i] = A[i] + B[i]
    end
end

A = zeros(10) .+ 3.0;
B = ones(10) .* 2;
C = similar(B);
vadd!(C, A, B)

In [ ]:
# an example of vector addition kernel for Apple GPU

A_d = MtlArray(Float32.(A));
B_d = MtlArray(Float32.(B));
C_d = similar(B_d);

# WARNING: the kernel above is not suitable for launching on GPUs
@metal vadd!(C_d, A_d, B_d)

**NOTE**: the *performance is terrible* for the code above, because each thread on the GPU is performing the same loop! To split work among the GPU cores, we have to remove the loop over all elements and instead use special functions which, when called by each GPU thread, return a unique index. For Metal.jl we use `thread_position_in_grid_1d()`, while the corresponding function for CUDA.jl is `threadIdx()`.

![](https://enccs.github.io/julia-for-hpc/_images/MappingBlocksToSMs.png)

In [ ]:
# replacing for-loop with thread_position_in_grid_1d()
function vadd!(C, A, B)
    index = thread_position_in_grid_1d()
    @inbounds C[index] = A[index] + B[index]
    return
end

A = MtlArray(Float32.(ones(Float32, 2^9)*2));
B = MtlArray(Float32.(ones(Float32, 2^9)*3));
C = similar(A);

N = length(A)
println("length of matrix A = ", N)
@metal threads=N vadd!(C, A, B)

@assert all(Array(C) .== 5.0)

However, this implementation will not work for arrays that are larger than the maximum number of threads in a block - try increasing the size to `2^12`! 

Clearly, GPUs have a limited number of threads they can run on a single SM. To parallelise over multiple SMs we need to run a kernel with multiple blocks where we also take advantage of the `blockDim()` and `blockIdx()` functions (in the case of NVIDIA). Here's how it looks for CUDA.jl:

In [ ]:
# this example is for CUDA.jl (requires CUDA-compatible NVIDIA GPUs)
function vadd!(C, A, B)
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if i <= length(A)
        @inbounds C[i] = A[i] + B[i]
    end
    return
end

A, B = CuArray(ones(2^9)*2), CuArray(ones(2^9)*3);
C = similar(A);

nthreads = 256
# smallest integer larger than or equal to length(A)/threads
numblocks = cld(length(A), nthreads)
println("nthreads = ", nthreads, ", numblocks = ", numblocks)

# run using 256 threads
@cuda threads=nthreads blocks=numblocks vadd!(C, A, B)

@assert all(Array(C) .== 5.0)

**Restrictions in kernel programming**

Within kernels, most of the Julia language is supported with the exception of functionality that requires the Julia runtime library. This means one cannot allocate memory or perform dynamic function calls, both of which are easy to do accidentally!

# 4. Julia packages for machine learning

The `MLJ.jl` (https://github.com/alan-turing-institute/MLJ.jl) package provides a unified interface to common machine learning algorithms, which include `generalized linear models`, `decision trees`, and `clustering`.

`Flux.jl` (https://github.com/FluxML/Flux.jl) and `Knet.jl` (https://github.com/denizyuret/Knet.jl) are powerful packages for Deep Learning.

Packages such as `Metalhead.jl`(https://github.com/FluxML/Metalhead.jl), `ObjectDetector.jl`(https://github.com/r3tex/ObjectDetector.jl), and `TextAnalysis.jl`(https://github.com/JuliaText/TextAnalysis.jl) provide ready to use pre-trained models for common tasks.

`AlphaZero.jl`(https://github.com/jonathan-laurent/AlphaZero.jl) provides a high performance implementation of the reinforcement learning algorithms from AlphaZero.

`Turing.jl`(https://turinglang.org/stable/) is a best in class package for probabilistic programming.

More packages for AI & ML from Julia official website
- ML: https://juliapackages.com/c/machine-learning
- NLP: https://juliapackages.com/c/nlp
- Neural Networks: https://juliapackages.com/c/neural-networks
- Reinforcement Learning: https://juliapackages.com/c/reinforcement-learning
- Supervised Learning: https://juliapackages.com/c/supervised-learning

### Training a deep neural network to classify penguins using `Flux.jl`

Flux.jl comes “batteries-included” with many useful tools built in, but also enables the user to write own Julia code for DL components.
- Flux has relatively few explicit APIs for features like regularisation or embeddings.
- All of Flux is straightforward Julia code and it can be worth to inspect and extend it if needed.
- Flux works well with other Julia libraries, like dataframes, images and differential equation solvers. One can build complex data processing pipelines that integrate Flux models.

To train a model we need four things:
- A collection of data points that will be provided to the objective function.
- A objective (cost or loss) function, that evaluates how well a model is doing given some input data.
- The definition of a model and access to its trainable parameters.
- An optimiser that will update the model parameters appropriately.

In [ ]:
using Pkg

Pkg.add("Flux")
Pkg.add("MLJ")
Pkg.add("DataFrames")
Pkg.add("PalmerPenguins")
Pkg.add("StatsBase")

In [ ]:
using Flux
using MLJ: partition, ConfusionMatrix
using DataFrames
using PalmerPenguins

table = PalmerPenguins.load()
df = DataFrame(table)
dropmissing!(df)

In [ ]:
# Pre-processing of the Penguins dataset and making it suitable for training a network.

# select feature and label columns
X = select(df, Not([:species, :sex, :island]))
Y = df[:, :species]

# split into training and testing parts
(xtrain, xtest), (ytrain, ytest) = partition((X, Y), 0.8, shuffle=true, rng=123, multi=true)

# use single precision and transpose arrays
xtrain, xtest = Float32.(Array(xtrain)'), Float32.(Array(xtest)')

# one-hot encoding
ytrain = Flux.onehotbatch(ytrain, ["Adelie", "Gentoo", "Chinstrap"])
ytest = Flux.onehotbatch(ytest, ["Adelie", "Gentoo", "Chinstrap"])

# count penguin classes to see if it's balanced
sum(ytrain, dims=2)
sum(ytest, dims=2)

In [ ]:
# We define `model` to be the neural network.

n_features, n_classes, n_neurons = 4, 3, 10
model = Chain(
        Dense(n_features, n_neurons),
        BatchNorm(n_neurons, relu),
        Dense(n_neurons, n_classes),
        softmax)

In [ ]:
# Define a `loss` function which will be minimized during training.
# Herein we use cross-entropy loss function typically used for classification
loss(x, y) = Flux.crossentropy(model(x), y)

# Define another function presenting the accuracy of the model
# onecold (opposite to onehot) gives back the original representation
function accuracy(x, y)
    return sum(Flux.onecold(model(x)) .== Flux.onecold(y)) / size(y, 2)
end

# check accuracy before training
accuracy(xtrain, ytrain)
accuracy(xtest, ytest)

In [ ]:
using StatsBase: sample

# Instead of training the entire dataset, training data were divided into `minibatches` and 
# update network weights on each minibatch separately.
function create_minibatches(xtrain, ytrain, batch_size=32, n_batch=10)
    minibatches = Tuple[]
    for i in 1:n_batch
        randinds = sample(1:size(xtrain, 2), batch_size)
        push!(minibatches, (xtrain[:, randinds], ytrain[:,randinds]))
    end
    return minibatches
end

In [ ]:
# We define an anonymous `callback` function to pass into the training function to monitor the progress, 
# to select standard ADAM optimizer, and to extract parameters of the model.

callback = () -> @show(loss(xtrain, ytrain))
opt = ADAM()
θ = Flux.params(model)

In [ ]:
# Here is to create the `minibatches` to call the `create_minibatches` function defined above.
minibatches = create_minibatches(xtrain, ytrain)

# We run 100 epochs to train the model.
for i in 1:100
    # train on minibatches
    Flux.train!(loss, θ, minibatches, opt, cb = Flux.throttle(callback, 1));
end

In [ ]:
# check final accuracy

accuracy(xtrain, ytrain)
accuracy(xtest, ytest)

In [ ]:
# Finally we create a confusion matrix to quantify the performance of the model.

predicted_species = Flux.onecold(model(xtest), ["Adelie", "Gentoo", "Chinstrap"])
true_species = Flux.onecold(ytest, ["Adelie", "Gentoo", "Chinstrap"])
ConfusionMatrix()(predicted_species, true_species)